# Applied Hydraulics - Modular Version

This notebook demonstrates the use of the modularized hydraulics package.

In [ ]:
import math
from hydraulics.constants import MOLECULAR_MASSES
from hydraulics.hydraulics import gas_density, pressure_static, hydrostatic_pressure_kgs, hydraulic_press
from hydraulics.gas_properties import gas_volume, gas_mixture_viscosity, gas_viscosity_temp
from hydraulics.fluid_mechanics import equi_diameter_hull_tube, equi_diameter_tube_tube, \
    velocity_hull_tube, velocity_tube_tube, reynolds_tube_tube, regime_tube_tube
from hydraulics.geometry import hydr_r_ring, hydr_r_square, hydr_r_rectangle, hydr_r_triangle

## 1.1 Find the molecular mass and density of water gas

Find the molecular mass and density of water gas at t = 90 °C and $p_{abs}$ = 1.2 $kgf/cm^2$ (~0.12 MPa). Composition of water gas: $H_2$ - 50 %, $CO$ - 40 %, $N_2$ - 5 %, $CO_2$ - 5 % (by volume).

In [ ]:
# Calculate molecular mass of the mixture
molecular_mass = (0.5 * MOLECULAR_MASSES["H2"] + 
                  0.4 * MOLECULAR_MASSES["CO"] + 
                  0.05 * MOLECULAR_MASSES["N2"] + 
                  0.05 * MOLECULAR_MASSES["CO2"])
print(f"Molecular mass of water gas: {molecular_mass:.2f}")

# Pressure needs to be converted to absolute:
p = 1 + 200 / 101.3

# Calculate density using the modularized function
density = gas_density(molecular_mass, 90, p)
print(f"Density of water gas: {density:.2f} kg/m³")

## 1.3 Find the volumetric composition of combustion products

Composition of combustion products of 1 kg of coke gas (kg): $CO_2$ - 1.45, $N_2$ - 8.74, $H_2O$ - 1.92 %. Find the volumetric composition of combustion products.

In [ ]:
# Mass composition of combustion products:
components_mass = {
    "CO2": 1.45,
    "N2": 8.74,
    "H2O": 1.92
}

# Volumetric composition of combustion products:
components_volumes = {k: v / MOLECULAR_MASSES[k] for k, v in components_mass.items()}
total_volume = sum(components_volumes.values())
components_volumes = {k: v/total_volume for k, v in components_volumes.items()}

for k, v in components_volumes.items():
    print(f"Volume fraction of {k}: {v:.3f}")

## 1.4 Find the absolute pressure in a drying tower

The vacuum in the drying tower of a sulfuric acid plant is measured by a U-shaped draft gauge filled with sulfuric acid with a density of 1800 kg/$cm^3$. The gauge readings are 3 cm. What is the absolute pressure in the tower, expressed in Pa, if the barometric pressure is 750 mm Hg?

In [ ]:
# Calculate pressure change using the modularized function
pressure_change = pressure_static(1800, 0.03)

# Calculate absolute pressure
pressure_absolute = 750 * 133.4 - pressure_change
print(f"Absolute pressure in the drying tower: {pressure_absolute:.2f} Pa")

## 1.5 Find the height of liquid rise in a piezometer

A pressure gauge on a pipeline filled with liquid shows a pressure of 0.18 kgf/$cm^2$. To what height h above the pressure gauge connection point does the liquid in the pipeline rise in an open piezometer, if this liquid is: a) water, b) carbon tetrachloride.

In [ ]:
# Relationship between pressure and height of liquid rise:
# $\rho \cdot g \cdot h = p [Pa]$
# $p[Pa] = p[N/m^2] = 10^4 \cdot g \cdot p[kgf/cm^2]$
# Height of rise: $h = 10^4 \cdot p[kgf/cm^2] / \rho$

height_rho = 10**4 * 0.18
print(f"Height of water rise: {height_rho / 1000:.2f} m")
print(f"Height of carbon tetrachloride rise: {height_rho / 1594:.2f} m")

## 1.6 Find the number of bolts and pressure on the bottom of the tank

The height of the fuel oil level in the tank is 7.6 m. The relative density of fuel oil is 0.96. At a height of 800 mm from the bottom, there is a circular hatch with a diameter of 760 mm, the cover of which is attached with bolts with a diameter of 10 mm. Assuming the permissible tensile stress for bolts is 700 $kgf/cm^2$, determine the required number of bolts. Also determine the pressure of fuel oil on the bottom of the tank.

In [ ]:
import math

# Calculate pressure on the bottom of the tank using the modularized function
print(f"Pressure on the bottom of the tank: {hydrostatic_pressure_kgs(960, 7.6):.3f} kgf/cm²")

middle_window = 7.6 - 0.8
middle_window_pressure = hydrostatic_pressure_kgs(960, middle_window)
print(f"Average pressure on the cover: {middle_window_pressure:.3f} kgf/cm²")

window_area_cm2 = 3.1415 * 76**2 / 4
window_pressure_force = window_area_cm2 * middle_window_pressure
print(f"Force acting on the cover: {window_pressure_force:.1f} kgf")

one_bolt = 700 * 3.1415 * 1 **2 / 4
print(f"Force sustained by one bolt: {one_bolt:.1f} kgf")

n_bolts = math.ceil(window_pressure_force / one_bolt)
print(f"Required number of bolts: {n_bolts:.0f} pieces")

## 1.7 Find the force acting on the pressed body

A force of 589 N (60 kgf) acts on the small piston with a diameter of 40 mm of a manual hydraulic press. Neglecting losses, determine the force acting on the pressed body if the diameter of the large piston is 300 mm.

In [ ]:
# Calculate force using the modularized function
print(f"Force acting on the pressed body: {hydraulic_press(40, 300, 589):.0f} N, {hydraulic_press(40, 300, 60):.0f} kgf")

## 1.8 Find the kinematic viscosity coefficient

The dynamic viscosity coefficient of a liquid at 50 °C is 30 mPa·s. The relative density of the liquid is 0.9. Determine the kinematic viscosity coefficient.

In [ ]:
print(f"Kinematic viscosity: {30e-3 / 900:.2e} m²/s")

## 1.9 Find the dynamic viscosity coefficient

Find the dynamic viscosity coefficient at 20 °C and atmospheric pressure of a nitrogen-hydrogen mixture containing 75 % hydrogen and 25 % nitrogen (by volume).

In [ ]:
# Viscosity dependence on pressure:
viscosities_0 = [17e-6, 8.42e-6]
C = [114, 73]

viscosities = [gas_viscosity_temp(v, 273+20, c) for v, c in zip(viscosities_0, C)]
print(f"Viscosity of nitrogen at the given temperature: {viscosities[0]:.2e} Pa·s")
print(f"Viscosity of hydrogen at the given temperature: {viscosities[1]:.2e} Pa·s")

print(f"Viscosity of gas mixture: {gas_mixture_viscosity(viscosities, [0.25, 0.75], [28, 2]):.2e} Pa·s")

## 1.10 Find the dynamic viscosity coefficient at 90°C

It is known that the dynamic viscosity coefficient of linseed oil at 30 °C is 0.331 P, and at 50°C - 0.176 P. What will be the dynamic viscosity coefficient of this oil at 90°C? (Use the linearity rule, taking 100% glycerin as the standard liquid, for example).

In [ ]:
# This problem requires implementing the linearity rule which is not provided in the original notebook.
# The implementation would need to be added here.

## 1.11 Find the water velocity in the tubes of the cooler

The cooler consists of 19 tubes with a diameter of 20x2 mm. Water enters the tube space of the cooler through a pipeline with a diameter of 57x3.5 mm. The water velocity in the pipeline is 1.4 m/s. The water flows from bottom to top. Determine the water velocity in the tubes of the cooler.

In [ ]:
# From the continuity of flow, it follows that the water flow rate in the heat exchanger should be
# equal to the water flow rate in the pipeline.
# Consequently, the ratio of velocities will be determined by the ratio of pipeline cross-sections

S_ratio = (57 - 3.5 * 2) ** 2 / (19 * (20 - 2 * 2) ** 2)
print(f"Ratio of pipeline cross-section to total cross-section of tubes in heat exchanger: {S_ratio:.3f}")
print(f"Water velocity in heat exchanger tubes: {1.4 * S_ratio:.3f} m/s")

## 1.12 Find the nitrogen velocity in the heat exchanger tubes

Nitrogen in the amount of $6400 m^3/h$ (counting at 0 °C and 760 mm Hg) passes through the tubes of a heat exchanger consisting of 379 tubes with a diameter of 16x1.5 mm under a pressure of $p_{gauge} = 3 kgf/cm^2 (~ 0.3 MPa)$. Nitrogen enters the heat exchanger at a temperature of 120 °C and exits at 30 °C. Determine the nitrogen velocity in the heat exchanger tubes at the inlet and outlet.

In [ ]:
# Cross-section of heat exchanger tubes:
S = 379 * 3.1415 * (0.016 - 2 * 0.0015) ** 2 / 4
print(f"Cross-section of heat exchanger tubes: {S:.2e} m²")

# Nitrogen flow rate:
N2_v_in = gas_volume(6400, (100000 + 3*98100), 120, normal_pressure=133.4*760)
N2_v_out = gas_volume(6400, (100000 + 3*98100), 30, normal_pressure=133.4*760)
print(f"Nitrogen flow rate at inlet: {N2_v_in:.2e} m³/h")
print(f"Nitrogen flow rate at outlet: {N2_v_out:.2e} m³/h")

# Nitrogen velocity:
print(f"Nitrogen velocity at inlet: {N2_v_in /(3600 * S):.2f} m/s")
print(f"Nitrogen velocity at outlet: {N2_v_out /(3600 * S):.2f} m/s")

## 1.13 Find the gas and liquid velocities in the cooler

The cooler consists of two concentric steel tubes with diameters of 29x2.5 mm and 54x2.5 mm. Brine flows through the inner tube at 3.73 t/h with a density of 1150 $kg/m^3$. Gas flows in the inter-tube space at 160 kg/h under a pressure of $p_{abs} = 3 kgf/cm^2$ (~0.3 MPa) at an average temperature of 0 °C. The gas density at 0 °C and 760 mm Hg is 1.2 $kg/m^3$. Find the gas and liquid velocities in the cooler.

In [ ]:
# Cross-sectional areas of tubes:
outer = 3.1415 * ((0.054 - 0.005)**2 - 0.029**2) / 4
inner = 3.1415 * (0.029 - 0.005)**2 / 4
print(f"Cross-sectional area of inner tube: {inner:.2e} m²")
print(f"Cross-sectional area of outer tube: {outer:.2e} m²")
print()

# Liquid flow rate
vol_l = 3730 / (3600 * 1150)
print(f"Brine flow rate: {vol_l:.2e} m³/s")
vel_l = vol_l / inner
print(f"Brine velocity: {vel_l:.2f} m/s")
print()

# Gas flow rate
vol_g = (760 * 133.4 / (3e4 * 9.81)) * 160 / (3600 * 1.2)
print(f"Gas flow rate: {vol_g:.2e} m³/s")
vel_g = vol_g / outer
print(f"Gas velocity: {vel_g:.2f} m/s")

## 1.14 Find the required diameter of the outer tube

Determine the required diameter of the outer tube under the conditions of the previous problem if the gas flows at atmospheric pressure, but at the same velocity and the same mass flow rate.

In [ ]:
vol_g = 160 / (3600 * 1.2)
print(f"Gas flow rate: {vol_g:.2e} m³/s")

outer_new = vol_g / vel_g
print(f"Cross-sectional area of outer tube: {outer_new:.2e} m²")

outer_d = (outer_new * 4 / 3.1415 + 0.029**2) ** 0.5
print(f"Inner diameter of outer tube: {outer_d*1000:.0f} mm")

## 1.15 Calculate the hydraulic radius

Calculate the hydraulic radius in general form for a filled cross-section for a ring cross-section, square, rectangle, and equilateral triangle.

In [ ]:
# These functions are now in the geometry module
print("Hydraulic radius functions are available in the geometry module:")
print("- hydr_r_ring(r_inner, r_outer)")
print("- hydr_r_square(a)")
print("- hydr_r_rectangle(a, b)")
print("- hydr_r_triangle(a)")

## 1.16 Find the equivalent diameter

Determine the equivalent diameter of the inter-tube space of a shell-and-tube heat exchanger consisting of 61 tubes with a diameter of 38x2.5 mm. The inner diameter of the shell is 625 mm.

In [ ]:
# Calculate equivalent diameter using the modularized function
eq_diameter = equi_diameter_hull_tube(61, 0.038, 0.625)
print(f"Equivalent diameter: {eq_diameter:.4f} m")

## 1.17 Determine the flow regime

Determine the flow regime in the annular space of a "tube-in-tube" heat exchanger. The outer tube is 96x3.5 mm, the inner tube is 57x3 mm, the water flow rate is 3.6 m³/h, the average water temperature is 20 °C.

In [ ]:
# Determine flow regime using the modularized function
regime = regime_tube_tube(3.6, 0.057, 0.096, 1.2e-3, 1000)
print(f"Flow regime: {regime}")